In [13]:
!pip install scikit-learn tqdm

In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from io import BytesIO
from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, resnet50

from sklearn.ensemble import IsolationForest

In [15]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# =========================
# 1) CSV 로드
# =========================
csv_path = "/content/drive/MyDrive/daangn_list_detail.csv"  # 경로 맞게 수정
df = pd.read_csv(csv_path)

assert "image" in df.columns
assert "image_count" in df.columns

# 이미지 있는 행 / 없는 행 분리
has_img_mask = (df["image_count"] > 0) & df["image"].notna() & (df["image"].astype(str).str.len() > 0)
df_with_img = df[has_img_mask].copy()
df_no_img  = df[~has_img_mask].copy()

print("이미지 있는 행:", len(df_with_img))
print("이미지 없는 행:", len(df_no_img))

이미지 있는 행: 1053
이미지 없는 행: 6


In [17]:
# =========================
# 2) ResNet50 feature extractor (네 노트북 스타일 그대로)
# =========================

# 사전학습 백본 (top 제거, 동결) – Pneumonia 노트북과 동일 구조
base = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base.trainable = False

# 네가 예전에 했던 것처럼:
#   - x는 0~1로 정규화 했다가 → Lambda에서 *255 후 resnet50.preprocess_input 적용
# 여기서는 편의를 위해 PIL → numpy → 0~1 로 만든 뒤, 같은 방식 사용
feature_model = models.Sequential([
    layers.Input(shape=(256, 256, 3)),
    # Pneumonia 코드 주석: "x/255로 0~1이므로, 255를 곱해 원래 스케일로 되돌린 뒤 preprocess_input"
    layers.Lambda(lambda z: resnet50.preprocess_input(z * 255.0)),
    base,
    layers.GlobalAveragePooling2D()  # Flatten 대신 권장
])

feature_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda_1 (Lambda)               │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [19]:
# =========================
# 3) 이미지 로딩 & 전처리 함수
# =========================

def load_image_from_url(url, timeout=5):
    """
    URL에서 이미지를 다운로드해 256x256 RGB numpy array (0~1)로 반환.
    실패 시 None.
    """
    try:
      # HTML 엔티티로 들어온 &amp; 를 실제 & 로 변환
        url = url.replace("&amp;", "&")
        resp = requests.get(url, timeout=timeout)
        resp.raise_for_status()
        img = Image.open(BytesIO(resp.content)).convert("RGB")
        img = img.resize((256, 256))
        img_arr = np.array(img).astype("float32") / 255.0  # 0~1
        return img_arr
    except Exception as e:
        # print("이미지 로드 실패:", url, e)
        return None

In [20]:
# =========================
# 4) 모든 이미지에서 feature 추출 (멀티 URL 평균)
# =========================

features = []
valid_indices = []  # feature 추출 성공한 df_with_img 의 index

for idx, row in tqdm(df_with_img.iterrows(), total=len(df_with_img)):
    raw = str(row["image"])

    # 여러 URL이 | 로 구분되어 있음 → split
    # 끝에 | 가 붙어있을 수 있으므로 공백/빈 문자열 제거
    url_list = [u.strip() for u in raw.split("|") if u.strip()]

    img_features = []

    for url in url_list:
        img_arr = load_image_from_url(url)
        if img_arr is None:
            # 다운로드/로드 실패 → 나중에 따로 처리
            continue

        # (256, 256, 3) → (1, 256, 256, 3)
        img_batch = np.expand_dims(img_arr, axis=0)  # (1, 256, 256, 3)

        # feature 추출
        feat = feature_model.predict(img_batch, verbose=0)  # shape: (1, 2048)
        img_features.append(feat[0])

    # 이 매물의 이미지들 중 하나라도 성공했다면 → 평균 feature 사용
    if len(img_features) == 0:
        # 이 매물은 이미지가 있다고 되어 있지만 로딩이 전부 실패한 케이스 → 나중에 0.8로 처리
        continue

    rep_feature = np.mean(img_features, axis=0)  # (2048,)
    features.append(rep_feature)
    valid_indices.append(idx)

features = np.array(features)  # (N_valid, D)
print("feature 추출 성공한 매물 수:", features.shape[0])

100%|██████████| 1053/1053 [2:46:23<00:00,  9.48s/it]

feature 추출 성공한 매물 수: 1053


In [21]:
# =========================
# 5) 비지도 이상탐지 (IsolationForest)
# =========================

# contamination: 대략 몇 %를 이상으로 볼지 (예: 0.05 = 상위 5% 가장 이상)
iso = IsolationForest(
    n_estimators=200,
    contamination=0.05,
    random_state=42,
    n_jobs=-1
)
iso.fit(features)

# score_samples는 값이 클수록 "정상" → 음수로 바꿔서 값이 클수록 "이상"이 되게
raw_scores = -iso.score_samples(features)  # (N_valid, )

# 0.0 ~ 1.0 min-max 정규화
min_s, max_s = raw_scores.min(), raw_scores.max()
norm_scores = (raw_scores - min_s) / (max_s - min_s + 1e-8)

In [22]:
# =========================
# 6) image_anomaly_score 컬럼 생성/채우기
# =========================

df["image_anomaly_score"] = np.nan

# (1) 이미지 있고 + feature 추출 성공한 행 → 모델 점수 (멀티이미지 평균 기반)
for idx, score in zip(valid_indices, norm_scores):
    df.loc[idx, "image_anomaly_score"] = score

# (2) 이미지가 아예 없는 행 (image_count == 0 등) → 1.0 (가장 의심)
df.loc[df_no_img.index, "image_anomaly_score"] = 1.0

# (3) 이미지가 있지만 다운로드 실패해서 score 못 만든 행 → 중간 이상(예: 0.8) 부여
failed_mask = has_img_mask & df["image_anomaly_score"].isna()
df.loc[failed_mask, "image_anomaly_score"] = 0.8

# 최종적으로 0.0 ~ 1.0 범위 안인지 확인
print(df["image_anomaly_score"].min(), df["image_anomaly_score"].max())

0.0 1.0


In [23]:
# =========================
# 7) CSV 저장
# =========================

out_path = "/content/drive/MyDrive/daangn_list_detail_with_image_score.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("저장 완료:", out_path)

저장 완료: /content/drive/MyDrive/daangn_list_detail_with_image_score.csv
